# Grounding LLM statistics facts using Retrieval Augmented Generation (RAG)

In this notebook, we share promising, early research advancements that tackle the challenge of provenance around real-world statistical data. This notebook connects to DataGemma, the first [open model](https://huggingface.co/google/datagemma-rag-27b-it) designed to connect large language models with the extensive, real-world data housed within Google's [Data Commons](https://datacommons.org).

This established approach retrieves relevant information from Data Commons before the LLM generates text, providing it with a factual foundation for its response. This implementation is only possible because of Gemini 1.5 Pro’s long context window allowing us to append the user query with the Data Commons data. More technical details of this approach can be found in this [paper](https://datacommons.org/link/DataGemmaPaper).

This demo is based on a finetuned [Gemma 2](https://ai.google.dev/gemma/docs) 27B model.

Please read [Gemma Terms of Use](https://ai.google.dev/gemma/terms).

***Disclaimer:***

*You're accessing a very early version of DataGemma. It is meant for trusted tester use (primarily for academic and research use) and not yet ready for commercial or general public use. This version was trained on a very small corpus of examples and may exhibit unintended, and at times controversial or inflammatory behavior. Please anticipate errors and limitations as we actively develop this large language model interface.*

*Your feedback and evaluations are critical to refining DataGemma's performance and will directly contribute to its training process. Known limitations are detailed in the paper, and we encourage you to consult it for a comprehensive understanding of DataGemma's current capabilities.*

## Step 0: Setup

To run this colab, you will need to use the A100 GPU and High-RAM runtime in Colab. With this runtime configuration, the total runtime of this notebook would take about 20 minutes.

You also need authentication for model and data access:

*   **Hugging Face Token**. To obtain the token, login to your Hugging Face account [token settings](https://huggingface.co/settings/tokens) to create a new token. Copy this token and store it on the colab notebook `Secrets` section with Name `HF_TOKEN`.

*   **Data Commons API Key**. Register for an API key from Data Commons [API key portal](https://apikeys.datacommons.org), make sure to enable `Data Commons NL API`. Once you get the API key, store it on the colab notebook `Secrets` section with Name `DC_API_KEY`.

*   **Gemini 1.5 Pro API Key**. Register for an API key from [Google AI Studio](https://aistudio.google.com/app/apikey). Once you get the API key, store it on the colab notebook `Secrets` section with Name `GEMINI_API_KEY`

Toggle the "Notebook access" button to enable all these secrets.



Then install the required libraries.

In [ ]:
!pip install -q git+https://github.com/datacommonsorg/llm-tools
!pip install -q bitsandbytes accelerate

## Step 1: Load the model

This section loads the finetuned Gemma2 27B model from Huggingface and creates a transformer model wrapper than can be used in the Retrieval Augmented Generation (RAG) workflow. More technical details of this approach can be found in the [paper](https://datacommons.org/link/DataGemmaPaper).

In [ ]:
import torch

import data_gemma as dg

from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Initialize Data Commons API client
DC_API_KEY = userdata.get('DC_API_KEY')
dc = dg.DataCommons(api_key=DC_API_KEY)

# Get Gemini 1.5 Pro model
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
gemini_model = dg.GoogleAIStudio(model='gemini-1.5-pro', api_keys=[GEMINI_API_KEY])


# Get finetuned Gemma2 model from HuggingFace
HF_TOKEN = userdata.get('HF_TOKEN')

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = 'google/datagemma-rag-27b-it'
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)
datagemma_model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             quantization_config=nf4_config,
                                             torch_dtype=torch.bfloat16,
                                             token=HF_TOKEN)

# Build the LLM Model stub to use in RAG flow
datagemma_model_wrapper = dg.HFBasic(datagemma_model, tokenizer)

## Step 2: Pick or enter a query for RAG

You can select a query or enter your own query to test RAG.


In [ ]:

#@title Pick a query from a sample list{ run: "auto" }
QUERY = "Do the US states with high coal fired power also have high rates of COPD?" #@param ["Do the US states with high coal fired power also have high rates of COPD?", "Is obesity in America continuing to increase?", "Which US states have the highest percentage of uninsured children?", "Which US states have the highest cancer rates?", "How have CO2 emissions changed in France over the last 10 years?", "Which New Jersey schools have the highest student to teacher ratio?", "Show me a breakdown of income distribution for Seattle.", "Which New Jersey cities have the best commute times for workers?", "Does India have more people living in the urban areas or rural areas?  How does that vary by states?  Are the districts with the most urban population also located in the states with the most urban population?", "Can you find a district in India each where: 1. there are more muslims than hindus or christians or sikhs;  2. more christians than the rest;  3. more sikhs than the rest.", "What are some interesting trends in Sunnyvale spanning gender, age, race, immigration, health conditions, economic conditions, crime and education?", "Where are the most violent places in the world?", "Compare Cambridge, MA and Palo Alto, CA in terms of demographics, education, and economy stats.", "Give me some farming statistics about Kern county, CA.", "What is the fraction households below poverty status receive food stamps in the US?  How does that vary across states?", "Is there evidence that single-parent families are more likely to be below the poverty line compared to married-couple families in the US?", "What patterns emerge from statistics on safe birth rates across states in India?", "Are there significant differences in the prevalence of various types of disabilities (such as vision, hearing, mobility, cognitive) between Dallas and Houston?", "Are there states in the US that stand out as outliers in terms of the prevalence of drinking and smoking?", "Has the use of renewables increased globally?", "Has the average lifespan increased globally?"]

In [ ]:
#@title Use your own query (Please see disclaimer at the top)
QUERY = 'Do the US states with high coal fired power also have high rates of COPD?' #@param {type:"string"}

## Step 3: Run RAG and Print Output


In [ ]:
from IPython.display import Markdown
import textwrap

def display_chat(prompt, text):
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  text = text.replace('•', '  *')
  text = textwrap.indent(text, '> ', predicate=lambda _: True)
  formatted_text = "<font size='+1' color='teal'>🤖\n\n" + text + "\n</font>"
  return Markdown(formatted_prompt+formatted_text)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

ans = dg.RAGFlow(llm_question=datagemma_model_wrapper, llm_answer=gemini_model, data_fetcher=dc).query(query=QUERY)
Markdown(textwrap.indent(ans.answer(), '> ', predicate=lambda _: True))


display_chat(QUERY, ans.answer())

... [RAG] Calling FINETUNED model for DC questions
... calling HF Pipeline API ""
Your role is that of a Question Generator.  Give..."
... [RAG] Making DC Calls
... calling DC for table with "What is the rate of coal fired power in US?"
... calling DC for table with "What is highest rate of COPD in US states?"
... calling DC for table with "What is the rate of COPD in US?"
... calling DC for table with "What is the rate of coal fired power in US states?"
... calling DC for table with "How has the rate of COPD changed over time in US states?"
... calling DC for table with "How has the rate of coal fired power changed over time in US states?"
... calling DC for table with "What is the rate of COPD in US states?"
... [RAG] Calling UNTUNED model for final response
... calling AIStudio gemini-1.5-pro "Using statistics from the tables below, respond t..."


<font size='+1' color='brown'>🙋‍♂️<blockquote>Do the US states with high coal fired power also have high rates of COPD?</blockquote></font><font size='+1' color='teal'>🤖

> To answer this query, we need to compare coal-fired power generation and COPD rates for each state. 
> 
> [Table 4] provides data on net coal-fired power generation for each state in 2020. [Table 5] provides data on COPD rates for each state from 2014 to 2021. 
> 
> Let's focus on the top 10 states with the highest coal-fired power generation in 2020 [Table 4]:
> 
> 1. **Wyoming:** 57220.4734 kWh
> 2. **West Virginia:** 28115.9007 kWh
> 3. **North Dakota:** 32042.801 kWh
> 4. **Kentucky:** 9746.6756 kWh
> 5. **Nebraska:** 9734.179 kWh
> 6. **Missouri:** 8413.2103 kWh
> 7. **Montana:** 7903.9661 kWh
> 8. **Utah:** 7017.4985 kWh
> 9. **Indiana:** 7072.5945 kWh
> 10. **Arkansas:** 5088.5615 kWh
> 
> Now let's see their corresponding COPD rates in 2020 [Table 5]:
> 
> 1. **Wyoming:** 6.0342% 
> 2. **West Virginia:** 9.9151% 
> 3. **North Dakota:** 5.4658%
> 4. **Kentucky:** 8.9724% 
> 5. **Nebraska:** 5.0078%
> 6. **Missouri:** 7.1813% 
> 7. **Montana:** 5.9228% 
> 8. **Utah:** 4.7961% 
> 9. **Indiana:** 7.4786% 
> 10. **Arkansas:** 7.7975%
> 
> **Observations:**
> 
> * States like West Virginia, Kentucky, Indiana, and Arkansas have both high coal-fired power generation and high COPD rates.
> * However, states like Wyoming, North Dakota, Nebraska, and Utah have high coal-fired power generation but relatively lower COPD rates. 
> 
> **Conclusion:**
> 
> While some states with high coal-fired power generation also exhibit high COPD rates, there isn't a simple direct correlation. Other factors likely contribute to COPD prevalence, including smoking rates, air pollution from other sources, and access to healthcare. 
> 
> 
> #### TABLES ####
> Table 1: Net Generation, Coal, All Sectors, Annual in United States (unit: kWh), according to eia.gov
> date | Net generation, coal, all sectors, annual
> ------------------------------------------------
> 2020 | 2348.536
> 2019 | 2938.985
> 2018 | 3516.992
> 2017 | 3708.8687
> 2016 | 3835.5215
> 2015 | 4216.5069
> 2014 | 4967.8966
> 2013 | 5002.5786
> 2012 | 4823.6722
> 2011 | 5563.2924
> 2010 | 5971.9631
> 2009 | 5723.8175
> 2008 | 6530.2225
> 2007 | 6694.0461
> 2006 | 6671.0628
> 2005 | 6811.3705
> 2004 | 6756.3687
> 2003 | 6803.4567
> 2002 | 6721.005
> 2001 | 6681.275
> 
> 
> Table 2: Chronic Obstructive Pulmonary Disease in States of United States (2020 to 2021) (unit: %), according to cdc.gov
> rank | place | Chronic Obstructive Pulmonary Disease
> ----------------------------------------------------
> 1 | West Virginia | 9.6571
> 2 | Kentucky | 8.5592
> 3 | Tennessee | 7.8078
> 4 | Louisiana | 7.7418
> 5 | Arkansas | 7.6916
> 6 | Ohio | 7.67
> 7 | Mississippi | 7.5369
> 8 | Indiana | 7.505
> 9 | Alabama | 7.302
> 10 | Missouri | 7.2912
> 
> 
> Table 3: Chronic Obstructive Pulmonary Disease in United States (unit: %), according to cdc.gov
> date | Chronic Obstructive Pulmonary Disease
> --------------------------------------------
> 2021 | 5.7
> 2020 | 5.6
> 2018 | 6.2
> 
> 
> Table 4: Net Generation, Coal, All Sectors, Annual in States of United States (2020) (unit: kWh), according to eia.gov
> place | variable | data
> -----------------------
> Alabama | Net generation, coal, all sectors, annual | 4415.2052
> Alaska | Net generation, coal, all sectors, annual | 1017.6891
> Arizona | Net generation, coal, all sectors, annual | 1852.3798
> Arkansas | Net generation, coal, all sectors, annual | 5088.5615
> California | Net generation, coal, all sectors, annual | 7.3738
> Colorado | Net generation, coal, all sectors, annual | 3353.8821
> Connecticut | Net generation, coal, all sectors, annual | -5.3645
> Delaware | Net generation, coal, all sectors, annual | 103.6413
> District of Columbia | Net generation, coal, all sectors, annual | N/A
> Florida | Net generation, coal, all sectors, annual | 766.9396
> Georgia | Net generation, coal, all sectors, annual | 1300.7158
> Hawaii | Net generation, coal, all sectors, annual | 828.1036
> Idaho | Net generation, coal, all sectors, annual | 10.7719
> Illinois | Net generation, coal, all sectors, annual | 2463.8986
> Indiana | Net generation, coal, all sectors, annual | 7072.5945
> Iowa | Net generation, coal, all sectors, annual | 4474.708
> Kansas | Net generation, coal, all sectors, annual | 5820.5127
> Kentucky | Net generation, coal, all sectors, annual | 9746.6756
> Louisiana | Net generation, coal, all sectors, annual | 843.5718
> Maine | Net generation, coal, all sectors, annual | 44.046
> Maryland | Net generation, coal, all sectors, annual | 554.7671
> Massachusetts | Net generation, coal, all sectors, annual | N/A
> Michigan | Net generation, coal, all sectors, annual | 2809.7405
> Minnesota | Net generation, coal, all sectors, annual | 2485.0872
> Mississippi | Net generation, coal, all sectors, annual | 1548.3168
> Missouri | Net generation, coal, all sectors, annual | 8413.2103
> Montana | Net generation, coal, all sectors, annual | 7903.9661
> Nebraska | Net generation, coal, all sectors, annual | 9734.179
> Nevada | Net generation, coal, all sectors, annual | 622.3837
> New Hampshire | Net generation, coal, all sectors, annual | 94.0383
> New Jersey | Net generation, coal, all sectors, annual | 103.2065
> New Mexico | Net generation, coal, all sectors, annual | 6071.3426
> New York | Net generation, coal, all sectors, annual | 7.431
> North Carolina | Net generation, coal, all sectors, annual | 1961.5495
> North Dakota | Net generation, coal, all sectors, annual | 32042.801
> Ohio | Net generation, coal, all sectors, annual | 3848.8873
> Oklahoma | Net generation, coal, all sectors, annual | 1491.5433
> Oregon | Net generation, coal, all sectors, annual | 384.3315
> Pennsylvania | Net generation, coal, all sectors, annual | 1863.4167
> Puerto Rico | Net generation, coal, all sectors, annual | N/A
> Rhode Island | Net generation, coal, all sectors, annual | N/A
> South Carolina | Net generation, coal, all sectors, annual | 2387.696
> South Dakota | Net generation, coal, all sectors, annual | 1846.2985
> Tennessee | Net generation, coal, all sectors, annual | 2068.4791
> Texas | Net generation, coal, all sectors, annual | 2684.7152
> Utah | Net generation, coal, all sectors, annual | 7017.4985
> Vermont | Net generation, coal, all sectors, annual | N/A
> Virginia | Net generation, coal, all sectors, annual | 438.015
> Washington | Net generation, coal, all sectors, annual | 672.0405
> West Virginia | Net generation, coal, all sectors, annual | 28115.9007
> Wisconsin | Net generation, coal, all sectors, annual | 4076.1819
> Wyoming | Net generation, coal, all sectors, annual | 57220.4734
> 
> 
> Table 5: Chronic Obstructive Pulmonary Disease (unit: %), according to cdc.gov
> date | Alabama | Alaska | Arizona | Arkansas | California | Colorado | Connecticut | Delaware | District of Columbia | Florida | Georgia | Hawaii | Idaho | Illinois | Indiana | Iowa | Kansas | Kentucky | Louisiana | Maine | Maryland | Massachusetts | Michigan | Minnesota | Mississippi | Missouri | Montana | Nebraska | Nevada | New Hampshire | New Jersey | New Mexico | New York | North Carolina | North Dakota | Ohio | Oklahoma | Oregon | Pennsylvania | Rhode Island | South Carolina | South Dakota | Tennessee | Texas | Utah | Vermont | Virginia | Washington | West Virginia | Wisconsin | Wyoming
> -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
> 2021 | 7.302 | 6.0817 | 5.8253 | 7.6916 | 4.9821 | 5.1472 | 5.3503 | 5.8549 | 4.6658 |  | 6.1714 | 3.8794 | 6.0132 | 5.7013 | 7.505 | 6.0833 | 5.974 | 8.5592 | 7.7418 | 6.9312 | 4.9971 | 5.615 | 6.9539 | 4.6899 | 7.5369 | 7.2912 | 5.8638 | 5.395 | 6.5995 | 5.914 | 5.0972 | 5.8554 | 5.7765 | 6.3108 | 5.3394 | 7.67 | 7.2801 | 5.9295 | 6.3907 | 5.6575 | 6.4553 | 5.7036 | 7.8078 | 6.2384 | 4.9599 | 5.6352 | 5.8187 | 5.1331 | 9.6571 | 5.2912 | 6.2802
> 2020 | 7.7289 | 5.46 | 5.9513 | 7.7975 | 5.329 | 5.0064 | 5.3593 | 6.0939 | 4.1492 | 6.9057 | 6.3432 | 3.9161 | 5.8348 | 5.5743 | 7.4786 | 5.7852 | 5.6744 | 8.9724 | 7.3701 | 7.1699 | 4.998 | 4.9913 | 6.9004 | 4.487 | 7.1294 | 7.1813 | 5.9228 | 5.0078 | 6.465 | 5.521 | 5.0041 | 5.429 | 5.4751 | 6.0331 | 5.4658 | 7.9603 | 7.6301 | 5.6906 | 6.6151 | 6.3488 | 6.6951 | 5.4311 | 7.7181 | 5.9148 | 4.7961 | 5.6266 | 5.5442 | 5.1441 | 9.9151 | 5.3852 | 6.0342
> 2018 | 8.2006 | 6.3722 | 6.4581 | 8.1042 | 5.3041 | 4.9572 | 5.5282 | 6.6358 | 5.35 | 7.3526 | 7.0336 | 4.3386 | 6.1369 | 6.3996 | 8.3839 | 5.8288 | 6.2735 | 9.6728 | 8.4607 | 6.813 | 5.7739 | 5.4893 | 7.9249 | 4.4406 | 8.2959 | 7.5731 | 5.9336 | 5.8168 | 7.1508 | 6.6299 | 5.5678 | 6.0615 | 6.4033 | 7.0272 | 5.589 | 8.0435 | 7.774 | 6.4748 | 6.7851 | 6.8116 | 6.9505 | 5.3219 | 8.5731 | 6.2627 | 4.8334 | 5.9171 | 6.0721 | 5.2125 | 10.822 | 5.7118 | 6.3916
> 2017 | 8.3059 | 4.8 | 6.1673 | 7.3185 | 5.0584 | 4.884 | 6.4893 | 8.2 | 5.4 | 7.7177 | 6.9828 | 3.6 | 5.1086 | 6.6115 | 8.1239 | 6.3253 | 6.068 | 9.005 | 7.8895 | 6 | 7.4 | 6.55 | 8.361 | 4.4791 | 7.4898 | 7.8648 | 5.6545 | 5.5118 | 7.1132 | 6.8018 | 7.2777 | 5.1594 | 5.8057 | 6.4583 | 5.1 | 9.0961 | 7.6392 | 5.2816 | 7.4733 | 7.147 | 6.277 | 4.8188 | 8.4465 | 5.6736 | 4.8732 | 6.1 | 6.6287 | 5.4563 | 9.6 | 5.832 | 5.8
> 2016 | 7.9354 | 5.4 | 6.1259 | 7.4885 | 5.32 | 4.8841 | 6.6088 | 7.6 | 5.5 | 7.0274 | 7.0388 | 3.9 | 5.3903 | 6.1476 | 7.5212 | 5.8964 | 6.0396 | 8.8365 | 7.2624 | 6.2 | 7.4 | 6.6496 | 8.8386 | 4.4806 | 7.601 | 7.6386 | 5.9688 | 5.6184 | 6.7245 | 6.7892 | 7.5322 | 5.3767 | 6.2125 | 5.9905 | 4.9 | 8.8549 | 7.0475 | 5.8207 | 7.296 | 7.5307 | 6.5656 | 5.627 | 8.4812 | 5.8098 | 5.0269 | 6 | 6.0687 | 5.6983 | 9.3 | 6.3976 | 6
> 2015 | 8.0367 | 4.9 | 6.2763 | 7.0993 | 4.9888 | 5.083 | 6.4341 | 7.5 | 5.3 | 6.6414 | 6.893 | 4 | 5.314 | 5.8449 | 7.3449 | 6.0286 | 5.9755 | 8.9068 | 6.9455 | 6.5 | 7.7 | 6.7567 | 8.1416 | 4.7548 | 7.0749 | 7.2898 | 5.566 | 5.4486 | 6.8516 | 6.8112 | 6.6502 | 5.5384 | 5.8924 | 6.0714 | 5.1 | 8.8204 | 6.961 | 5.4328 | 7.4508 | 6.848 | 6.3354 | 5.2279 | 8.2169 | 5.9321 | 4.8667 | 5.8 | 5.9793 | 5.5045 | 8.5 | 6.1144 | 6.6
> 2014 | 8.3481 | 5 | 6.6436 | 7.6951 | 5.4254 | 4.847 | 6.4958 | 7.2 | 5.1 | 6.8535 | 6.9082 | 3.7 | 5.335 | 5.9973 | 8.085 | 5.7719 | 6.3163 | 9.112 | 7.058 | 6.4 | 8 | 7.4905 | 8.5497 | 4.7214 | 7.2584 | 7.4483 | 5.9609 | 5.6251 | 7.2217 | 6.7471 | 7.1785 | 5.3928 | 6.1946 | 6.2976 | 4.5 | 9.3054 | 7.1848 | 5.6487 | 7.4678 | 6.6089 | 6.5081 | 5.4514 | 8.3177 | 5.7875 | 4.6279 | 5.9 | 6.2613 | 5.7134 | 8.5 | 5.9432 | 6.4
> 
> 
> Table 6: Net Generation, Coal, All Sectors, Annual (unit: kWh), according to eia.gov
> date | Alabama | Alaska | Arizona | Arkansas | California | Colorado | Connecticut | Delaware | Florida | Georgia | Hawaii | Idaho | Illinois | Indiana | Iowa | Kansas | Kentucky | Louisiana | Maine | Maryland | Massachusetts | Michigan | Minnesota | Mississippi | Missouri | Montana | Nebraska | Nevada | New Hampshire | New Jersey | New Mexico | New York | North Carolina | North Dakota | Ohio | Oklahoma | Oregon | Pennsylvania | South Carolina | South Dakota | Tennessee | Texas | Utah | Virginia | Washington | West Virginia | Wisconsin | Wyoming
> -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
> 2020 | 4415.2052 | 1017.6891 | 1852.3798 | 5088.5615 | 7.3738 | 3353.8821 | -5.3645 | 103.6413 | 766.9396 | 1300.7158 | 828.1036 | 10.7719 | 2463.8986 | 7072.5945 | 4474.708 | 5820.5127 | 9746.6756 | 843.5718 | 44.046 | 554.7671 |  | 2809.7405 | 2485.0872 | 1548.3168 | 8413.2103 | 7903.9661 | 9734.179 | 622.3837 | 94.0383 | 103.2065 | 6071.3426 | 7.431 | 1961.5495 | 32042.801 | 3848.8873 | 1491.5433 | 384.3315 | 1863.4167 | 2387.696 | 1846.2985 | 2068.4791 | 2684.7152 | 7017.4985 | 438.015 | 672.0405 | 28115.9007 | 4076.1819 | 57220.4734
> 2019 | 5430.9816 | 931.0963 | 3184.1024 | 7717.0859 | 6.0971 | 4397.0764 | 14.877 | 121.9658 | 988.6534 | 2375.3538 | 919.7034 | 11.7427 | 3851.6031 | 9027.1454 | 7013.3025 | 5944.6517 | 11563.1575 | 1592.6063 | 52.7977 | 944.9408 |  | 3739.8354 | 3156.8033 | 1482.204 | 9068.726 | 13176.6738 | 10548.3435 | 885.0403 | 252.0721 | 117.1407 | 6997.2505 | 21.6795 | 2920.7675 | 32932.6266 | 3998.1619 | 1975.9397 | 609.3664 | 2963.4563 | 2881.418 | 2950.0526 | 2736.6441 | 3167.5368 | 7879.4904 | 399.4687 | 941.6967 | 32408.4154 | 4521.4223 | 60980.6111
> 2018 | 6496.3076 | 853.3038 | 4291.4077 | 9958.3324 | 7.1335 | 4630.7429 | 92.2353 | 282.7222 | 1424.244 | 3059.1658 | 921.3696 | 11.608 | 4687.1042 | 11563.1034 | 9064.7318 | 7029.0952 | 13253.7018 | 2526.9268 | 52.8517 | 1666.1153 |  | 4238.4752 | 4181.9133 | 1770.1165 | 10342.0307 | 12582.2186 | 12103.3825 | 819.9509 | 487.2886 | 134.202 | 6400.7193 | 35.3245 | 3049.6946 | 36234.7612 | 5027.6094 | 3780.0919 | 352.9049 | 3441.7922 | 3829.2666 | 2660.1932 | 3093.2659 | 3903.0339 | 8212.7472 | 1088.7513 | 715.1606 | 34352.4086 | 5736.0216 | 68524.3293
> 2017 | 6445.3612 | 749.9589 | 4454.561 | 8750.5671 | 7.3998 | 5205.6302 | 55.2422 | 374.5499 | 1788.273 | 3118.6485 | 964.9037 | 13.8605 | 4536.8366 | 10865.2383 | 8066.2771 | 6661.1097 | 12846.0 | 2635.1705 | 50.8596 | 1412.3666 | 165.5581 | 4211.8465 | 4090.6339 | 1547.554 | 11048.0646 | 13155.5169 | 11045.1669 | 627.7021 | 212.7385 | 136.8217 | 8798.7841 | 39.3132 | 3353.5401 | 35356.8389 | 5858.5083 | 4415.2545 | 416.6825 | 3722.9751 | 3610.777 | 2360.5207 | 4127.5241 | 4759.4021 | 8503.0538 | 1266.2506 | 739.0334 | 37581.5958 | 6188.73 | 69084.45
> 2016 | 7039.1708 | 800.1063 | 4377.8851 | 7955.0609 | 8.1451 | 5402.1497 | 49.5472 | 504.5846 | 1911.5245 | 3675.6742 | 1047.9356 | 17.3323 | 4627.9168 | 10927.1158 | 8042.3422 | 7928.6805 | 15048.976 | 2566.2809 | 52.4061 | 2301.6782 | 274.6235 | 4071.3679 | 4199.9596 | 1786.2753 | 9902.8217 | 13692.2707 | 11485.922 | 742.1784 | 314.0723 | 148.1409 | 8776.4813 | 90.1509 | 3683.9706 | 35153.8916 | 5908.4888 | 4877.125 | 463.7372 | 4275.104 | 4231.8723 | 2411.3666 | 4685.9623 | 4343.0055 | 8520.8244 | 1960.097 | 630.3768 | 39026.1925 | 5776.9452 | 68393.597
> 2015 | 8529.7858 | 904.0116 | 5293.1618 | 7295.9692 | 7.6566 | 5782.7279 | 167.3286 | 635.3999 | 2121.9533 | 3646.2087 | 939.8546 | 47.8021 | 5736.8652 | 11832.621 | 9547.0563 | 8449.2358 | 16396.1082 | 3249.4031 | 69.1456 | 2325.3801 | 331.3968 | 5323.2798 | 4503.4808 | 2140.1709 | 10752.4447 | 15524.269 | 12782.5934 | 926.3588 | 700.3186 | 198.3127 | 9779.3481 | 121.0778 | 3977.4136 | 36708.213 | 6169.9868 | 6359.0122 | 591.5975 | 5053.796 | 4622.3745 | 1749.3204 | 4637.5572 | 4425.2197 | 10610.0729 | 2059.3763 | 704.8573 | 36920.1495 | 6451.6776 | 73485.6713
> 2014 | 9765.5232 | 757.4428 | 6336.8209 | 11190.1151 | 20.8559 | 6080.1524 | 229.4264 | 927.3984 | 2621.8396 | 4497.4874 | 1067.7216 | 47.5855 | 6773.9051 | 14789.0324 | 10844.3063 | 9908.2217 | 18927.3517 | 4137.1665 | 59.4223 | 2953.5406 | 413.1478 | 5326.1849 | 5127.1588 | 3590.5594 | 11950.4305 | 15234.253 | 13256.7933 | 2322.8147 | 982.5684 | 284.0902 | 9738.4369 | 233.6515 | 4954.8893 | 37082.3517 | 7743.8062 | 7709.3349 | 805.1029 | 6174.4222 | 5990.2957 | 3165.012 | 5481.5403 | 5495.4278 | 11359.0787 | 2503.6089 | 952.1642 | 41886.9251 | 6509.2941 | 74437.0764
> 2013 | 9738.1034 | 846.7987 | 6555.2989 | 10771.7197 | 21.5187 | 6394.3999 | 189.2914 | 1671.6633 | 2370.2597 | 4033.1307 | 996.4026 | 56.8332 | 6818.2889 | 14104.084 | 10763.7507 | 10284.7667 | 18902.8903 | 4505.7355 | 47.2335 | 2622.4392 | 589.5958 | 5677.5166 | 4343.3067 | 2910.2888 | 12593.9031 | 14672.5614 | 14346.0793 | 1892.199 | 1102.6611 | 228.2298 | 11537.1226 | 239.3248 | 4780.4979 | 37997.456 | 8166.3558 | 7784.1975 | 957.9231 | 6920.6891 | 5120.593 | 3381.0502 | 5000.1717 | 5642.2296 | 11827.4031 | 2563.0884 | 967.5827 | 38972.1833 | 7083.4953 | 79703.2004
> 2012 | 9468.6328 | 937.2602 | 6118.6028 | 9628.2167 | 36.2393 | 6646.7617 | 181.5988 | 1553.9864 | 2294.392 | 4111.1446 | 1101.8789 | 48.0836 | 6273.8955 | 14139.9828 | 11482.9842 | 9695.8761 | 18861.4892 | 4654.8364 | 34.0772 | 2748.5975 | 320.6536 | 5368.2441 | 4225.5276 | 2416.3959 | 12076.781 | 13929.3343 | 13496.9721 | 1486.3104 | 957.0009 | 214.5089 | 11971.964 | 232.5 | 5222.8437 | 40178.4097 | 7409.6486 | 7671.9828 | 675.4527 | 6824.9171 | 6017.3728 | 3500.2992 | 5486.473 | 5293.9575 | 10791.0108 | 1732.0148 | 545.4668 | 37830.1326 | 5726.1732 | 75281.9117
> 2011 | 11835.748 | 908.6816 | 6751.041 | 10002.6574 | 52.6668 | 6629.3146 | 146.52 | 1603.1423 | 2728.3813 | 6136.4352 | 1032.2375 | 52.65 | 6995.2474 | 15981.0654 | 12465.6675 | 11031.154 | 20969.9994 | 5381.601 | 41.5261 | 3605.8334 | 613.6489 | 5964.5809 | 5285.3253 | 3263.7231 | 13028.3612 | 15093.1836 | 14104.5388 | 1993.0251 | 1672.3188 | 470.6431 | 13044.0583 | 483.3948 | 6186.8615 | 39544.709 | 9123.4518 | 9100.1712 | 860.7794 | 7892.2626 | 7312.4485 | 3138.8121 | 6371.1446 | 6156.8895 | 11772.6654 | 2453.7202 | 765.8149 | 40915.5779 | 6999.4492 | 72179.9089
> 2010 | 13175.2641 | 868.9901 | 6811.5307 | 9634.6558 | 56.2767 | 6846.7604 | 727.6538 | 2854.8642 | 3178.233 | 7547.0 | 1133.071 | 56.1986 | 7290.2953 | 17306.3255 | 13531.7557 | 11372.2981 | 20939.3152 | 5264.1739 | 65.6748 | 4088.6316 | 1264.9001 | 6641.7343 | 5287.6857 | 4587.9378 | 12516.0298 | 18774.6755 | 12769.3981 | 2589.2124 | 2340.9981 | 729.3513 | 12408.0284 | 700.1442 | 7514.812 | 42181.4836 | 10210.8869 | 8371.0159 | 1075.2606 | 8682.6974 | 8126.0503 | 4041.024 | 6871.2253 | 5949.3481 | 12271.062 | 3172.8824 | 1264.559 | 42144.7652 | 7058.8638 | 76145.5499
> 2009 | 11687.568 | 902.2695 | 6259.7908 | 8656.0612 | 55.4621 | 6362.5869 | 688.835 | 3193.9835 | 2895.1966 | 7221.6306 | 1113.9429 | 53.1155 | 7030.4306 | 16768.2688 | 12315.5413 | 11382.4259 | 19466.332 | 5135.5619 | 54.2619 | 4216.5287 | 1385.1866 | 6751.2063 | 5553.1336 | 4379.4025 | 12013.1157 | 15865.4112 | 12881.3369 | 2808.5204 | 2192.5872 | 582.4692 | 14295.6008 | 660.8395 | 6887.3832 | 44523.8952 | 9863.2165 | 9161.6536 | 839.3902 | 8326.8112 | 7511.6501 | 3986.4754 | 6602.1432 | 5608.7387 | 13044.6693 | 3229.8122 | 1121.5382 | 36844.2585 | 6575.8085 | 74938.2717
> 2008 | 15812.0794 | 898.8179 | 6980.5308 | 9084.7673 | 62.2986 | 7122.5906 | 1237.2049 | 5958.8978 | 3498.8041 | 8994.4944 | 1236.7331 | 58.9509 | 7581.6859 | 18994.5168 | 13395.3166 | 12109.0961 | 21357.3878 | 5433.339 | 264.4096 | 4787.7584 | 1643.0271 | 7022.8426 | 6052.0571 | 5659.5998 | 12412.6777 | 18774.3249 | 11957.2677 | 2944.0248 | 2622.3527 | 1036.4365 | 13435.4919 | 996.965 | 8143.8533 | 45124.1319 | 11349.533 | 9897.8345 | 1073.12 | 9322.9269 | 9171.9433 | 4580.6187 | 9133.084 | 6052.5569 | 14277.1132 | 4056.4718 | 1335.1624 | 48423.0809 | 7393.3505 | 80227.9364
> 2007 | 16690.869 | 942.8331 | 6692.2012 | 9037.3264 | 63.401 | 7480.5842 | 1059.9479 | 6448.9004 | 3697.1199 | 9657.5021 | 1200.0922 | 55.5205 | 7503.6168 | 19249.3129 | 12665.1822 | 13021.9335 | 21256.761 | 5268.1421 | 283.0186 | 5253.3244 | 1869.5851 | 7080.1508 | 6181.8932 | 5944.414 | 12752.9045 | 19028.3615 | 11006.7685 | 2726.1493 | 2992.2252 | 1176.6312 | 13870.6917 | 1118.8149 | 8771.9581 | 44673.0552 | 11576.1097 | 9475.6826 | 1169.0318 | 9765.4974 | 9356.8047 | 3354.2906 | 9753.8849 | 6179.884 | 14308.864 | 4569.8292 | 1324.2593 | 50089.2367 | 7134.1748 | 80629.5463
> 2006 | 16873.8364 | 914.2858 | 6707.8967 | 8570.1978 | 62.0563 | 7683.5116 | 1217.218 | 5782.6098 | 3601.2205 | 9447.9853 | 1182.3765 | 56.0384 | 7248.4253 | 19525.0165 | 11535.1327 | 12045.679 | 21614.914 | 5669.7895 | 242.7148 | 5225.8938 | 1737.628 | 6753.6482 | 6404.5897 | 6232.5199 | 13255.7808 | 17933.4769 | 11667.304 | 2875.3485 | 2969.6314 | 1254.0208 | 15217.5959 | 1097.5266 | 8465.2596 | 44468.7601 | 11618.995 | 9747.119 | 645.7923 | 9796.1613 | 9057.8779 | 4234.7015 | 9935.9781 | 6266.8653 | 14593.3272 | 4468.2525 | 1000.3249 | 50042.5188 | 7192.3578 | 82063.3916
> 2005 | 17096.6721 | 936.0845 | 6874.9794 | 8283.4736 | 59.6008 | 7679.4031 | 1139.5899 | 5718.3267 | 3503.8207 | 9767.3113 | 1261.6086 | 66.6423 | 7316.8834 | 19563.7826 | 11554.3482 | 12559.9182 | 21297.7786 | 5044.6234 | 242.8531 | 5239.7722 | 1879.2756 | 6996.5195 | 6436.022 | 5724.7798 | 13373.4969 | 18958.9417 | 11818.454 | 7558.8734 | 3136.7055 | 1343.6724 | 15498.4479 | 1076.5724 | 9009.998 | 46833.7319 | 11935.9486 | 10231.8061 | 959.3944 | 9713.5222 | 9295.2376 | 3863.7963 | 9856.6039 | 6513.1803 | 14635.6866 | 4675.252 | 1677.1449 | 50215.548 | 7523.9929 | 84304.3577
> 2004 | 16516.4644 | 984.3675 | 7043.1194 | 9220.5302 | 62.9047 | 7835.6915 | 1217.0931 | 5709.94 | 3724.4094 | 9116.2807 | 1259.2575 | 71.2767 | 7498.2788 | 19359.9866 | 11941.9618 | 12651.2901 | 20771.4455 | 5196.9628 | 273.5886 | 5263.3496 | 1641.594 | 6822.8613 | 6682.8077 | 6049.7154 | 13052.7028 | 18688.2364 | 11693.9728 | 7781.559 | 3159.4517 | 1195.4845 | 15371.2449 | 1192.0102 | 8832.5468 | 43530.3345 | 11190.532 | 9588.4118 | 996.1334 | 9441.4961 | 9243.1609 | 4698.8834 | 9822.6202 | 6648.4104 | 15247.4013 | 4769.6674 | 1684.7271 | 48217.7744 | 7643.8117 | 85140.8945
> 2003 | 17030.4306 | 847.7053 | 6912.6847 | 8625.9645 | 65.9886 | 7974.7759 | 1205.4799 | 4922.3487 | 3979.9013 | 9119.8396 | 1314.0964 | 66.2251 | 7007.1127 | 19003.2537 | 12175.3763 | 12893.7071 | 20417.068 | 5062.7554 | 288.1961 | 5447.1653 | 1696.5947 | 6749.9708 | 7055.5102 | 5955.6297 | 12998.1688 | 18538.5807 | 12052.0274 | 7597.6428 | 3065.4637 | 1138.1357 | 15345.7834 | 1229.7216 | 8878.1504 | 46065.3223 | 11785.8886 | 10464.3242 | 1213.5302 | 9374.7944 | 9019.1191 | 4492.697 | 9391.7688 | 6671.9609 | 15244.3049 | 5035.0817 | 1816.7738 | 51023.1408 | 7613.738 | 84102.0949
> 2002 | 16080.4091 | 895.6171 | 7083.9186 | 8536.2133 | 66.7533 | 7880.8578 | 933.0547 | 4296.3262 | 3950.2026 | 9264.898 | 1246.9997 | 67.6476 | 6920.6786 | 19113.1142 | 12054.9547 | 13034.3589 | 20361.0736 | 4903.1792 | 466.0159 | 5277.5735 | 1792.503 | 6659.4887 | 6758.4476 | 5195.8029 | 11883.18 | 16823.9379 | 11539.6629 | 7550.4154 | 2933.1166 | 1123.051 | 14500.4848 | 1214.3043 | 9030.2577 | 46400.8286 | 11654.4845 | 10307.8112 | 1075.7836 | 9237.3817 | 8994.5389 | 4304.5222 | 10301.3329 | 6539.6549 | 14834.6096 | 5226.1623 | 1430.9824 | 51484.7653 | 7310.6152 | 83843.4713
> 2001 | 16160.2571 | 890.9271 | 7596.7894 | 9200.7497 | 64.7589 | 8123.2649 | 1088.2498 | 4233.3935 | 4171.1732 | 8900.6617 | 1308.9984 | 54.7622 | 6369.9393 | 18941.0995 | 11823.0322 | 11756.5124 | 22319.4769 | 4905.931 | 410.9942 | 5280.1936 | 1727.4094 | 6832.3836 | 6385.8049 | 6728.393 | 11655.2079 | 18783.4791 | 11767.8116 | 8452.6203 | 2952.0899 | 1097.3038 | 15506.001 | 1227.9055 | 8886.3195 | 45186.8129 | 10907.9502 | 9984.0697 | 1281.7571 | 9098.3593 | 9028.0564 | 4764.9254 | 10386.4065 | 6336.8816 | 14747.5968 | 5247.0147 | 1517.3371 | 44526.922 | 7629.8714 | 86502.991
> 

</font>

# More Information on Retrieval Augmented Generation (RAG)

Retrieval Augmented Generation (RAG): This established approach retrieves relevant information from Data Commons before the LLM generates text, providing it with a factual foundation for its response. This implementation is only possible because of Gemini 1.5 Pro’s long context window allowing us to append the user query with the Data Commons data.

Here's how RAG works:

1.   User Query: A user submits a query to the LLM.
2.   Query Analysis & Data Commons Query Generation: The DataGemma model (based on the Gemma 2 (27B) model and fully fine-tuned for this RAG task) analyzes the user's query and generates a corresponding query (or queries) in natural language that can be understood by Data Commons' existing natural language interface.
3.   Data Retrieval from Data Commons: Data Commons is queried using this natural language query, and relevant data tables, source information, and links are retrieved.
4.   Augmented Prompt: The retrieved information is added to the original user query, creating an augmented prompt.
5.   Final Response Generation: A larger LLM (Gemini 1.5 Pro) uses this augmented prompt, including the retrieved data, to generate a comprehensive and grounded response.

In the above example, 7 questions are asked of Data Commons (eg "What is the rate of coal fired power in US states?") and corresponding data tables are retrieved. The data in these table is used to compose the final response with coherent information and insight.